In [16]:
import requests
from datetime import datetime, timezone

# === User-set inputs ===
zomma_base_price  = 2392.02
deribit_base_price = 2328.0
size              = 1.0
expiry_date_str   = "09MAY25"
desired_hf        = 1.2

def compute_arbitrage(
    zomma_base_price: float,
    deribit_base_price: float,
    size: float,
    desired_hf: float,
    expiry_date_str: str
):
    # 1) fetch spot
    spot = requests.get(
        "https://api.coingecko.com/api/v3/simple/price",
        params={"ids": "bitcoin", "vs_currencies": "usd"}
    ).json()["bitcoin"]["usd"]

    # 2) times
    entry  = datetime.now(timezone.utc)
    expiry = datetime.strptime(expiry_date_str + " 08:00", "%d%b%y %H:%M") \
                     .replace(tzinfo=timezone.utc)

    # 3) fees
    zomma_fee = 0.0003 * spot + 0.01 * zomma_base_price
    deribit_fee = min(0.0003 * spot, 0.125 * deribit_base_price)

    # 4) profit (Zomma short, Deribit long)
    sell_proceeds = zomma_base_price - zomma_fee
    buy_cost      = deribit_base_price + deribit_fee
    profit        = sell_proceeds - buy_cost

    # 5) collateral & returns
    collateral     = 0.1 * desired_hf * (size * spot + zomma_base_price)
    roi            = profit / collateral
    days           = max((expiry - entry).days, 1)
    apr_linear     = roi * (365 / days)
    apy_compounded = (1 + roi) ** (365 / days) - 1

    return {
        "spot": spot,
        "deribit_fee": deribit_fee,
        "zomma_fee": zomma_fee,
        "entry_utc": entry,
        "expiry_utc": expiry,
        "profit": profit,
        "collateral": collateral,
        "roi": roi,
        "trade_days": days,
        "apr_linear": apr_linear,
        "apy_compounded": apy_compounded
    }

# === Compute & print ===
res = compute_arbitrage(
    zomma_base_price,
    deribit_base_price,
    size,
    desired_hf,
    expiry_date_str
)

print(f"BTC spot: {res['spot']:.2f} USD")
print(f"Deribit fee: {res['deribit_fee']:.2f} USD\n")
print(f"Zomma fee: {res['zomma_fee']:.2f} USD\n")
print("=== Arbitrage Summary ===")
print(f"Entry (UTC):          {res['entry_utc']:%Y-%m-%d %H:%M:%S}")
print(f"Expiry (UTC):         {res['expiry_utc']:%Y-%m-%d %H:%M:%S}")
print(f"Profit:               {res['profit']:.2f} USD")
print(f"Collateral required:  {res['collateral']:.2f} USD")
print(f"ROI:                  {res['roi']:.6%}")
print(f"Trade Days:           {res['trade_days']} days")
print(f"APR (linear):         {res['apr_linear']:.6%}")
print(f"APY (compounded):     {res['apy_compounded']:.6%}")


BTC spot: 94967.00 USD
Deribit fee: 28.49 USD

Zomma fee: 52.41 USD

=== Arbitrage Summary ===
Entry (UTC):          2025-04-29 10:12:32
Expiry (UTC):         2025-05-09 08:00:00
Profit:               -16.88 USD
Collateral required:  11683.08 USD
ROI:                  -0.144486%
Trade Days:           9 days
APR (linear):         -5.859704%
APY (compounded):     -5.695324%
